In [1]:
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import shapely
import time
import random
from shapely import wkt
from shapely.ops import snap
from shapely.geometry.point import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import pyproj
from plotly.graph_objs import Layout
from matplotlib.pyplot import plot

from string import digits
remove_digits = str.maketrans('', '', digits)

from rosreestr2coord import Area

from tqdm import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [3]:
def geoseries2shapedict(gdf, name_col, gdf_type):
    regions = {}
    for und, reg in tqdm(gdf.iterrows()):
        if type(reg.geometry) == MultiPolygon:
            x, y = np.array([[], []])
            for poly in reg.geometry.geoms:
                x_, y_ = poly.exterior.coords.xy
                x, y = (np.append(x, x_), np.append(y, y_))
                x, y = (np.append(x, None), np.append(y, None))
            x = x[:len(x)-1]
            y = y[:len(y)-1]
        elif type(reg.geometry) == Point:
            x, y = np.array(reg.geometry.coords.xy)
        else:
            x, y = np.array(reg.geometry.exterior.coords.xy)
        
        if gdf_type == 'rumap':
            regions[reg[name_col]] = x, y, reg.federal_district
        elif gdf_type == 'regmap':
            regions[reg[name_col]] = x, y, reg.index
        else:
            x, y = np.array(reg.geometry.exterior.coords.xy)
    return regions              

# climate map

In [4]:
# чтение файлов с полигонами зон
arctic = gpd.read_file('qgis_data/arctic_zone.shp') 
subarctic = gpd.read_file('qgis_data/subarctic_zone.shp') 
umerenny = gpd.read_file('qgis_data/umerenny_zone.shp')
subtropic = gpd.read_file('qgis_data/subtropic_zone.shp')

In [5]:
zones_list = [arctic, subarctic, umerenny, subtropic]
zone_names = ['Арктический', 'Субарктический', 'Умеренный', 'Субтропический']
zone_colors = ['mediumpurple', 'powderblue', 'yellowgreen', 'red']
zone_ids = [1,2,3,4]

In [7]:
climate_zones = gpd.GeoDataFrame()
for zone_ind in range(len(zones_list)):
    zone = zones_list[zone_ind][['geometry']]
    zone['level'] = zone_names[zone_ind]
    zone['zone_color'] = zone_colors[zone_ind]
    zone['zone_id'] = zone_ids[zone_ind]
    climate_zones = climate_zones.append(zone)
climate_zones.reset_index(inplace=True)
climate_zones = climate_zones.to_crs('ESRI:102027')

In [8]:
climate_zones

,index,geometry,level,zone_color,zone_id
0,0,"POLYGON ((-1413920.080 4921824.947, -1403713.4...",Арктический,mediumpurple,1
1,1,"POLYGON ((-1006376.684 4533175.724, -981872.89...",Арктический,mediumpurple,1
2,2,"POLYGON ((-864322.287 4210054.138, -863911.984...",Арктический,mediumpurple,1
3,0,"POLYGON ((-1414443.099 4921818.789, -1340204.9...",Субарктический,powderblue,2
4,1,"POLYGON ((-1006106.431 4532261.530, -903793.08...",Субарктический,powderblue,2
5,2,"POLYGON ((-1003981.176 3915719.965, -937153.43...",Субарктический,powderblue,2
6,3,"POLYGON ((-2380361.237 5084586.569, -2366569.0...",Субарктический,powderblue,2
7,0,"POLYGON ((3146486.373 4562285.823, 3135380.152...",Умеренный,yellowgreen,3
8,1,"POLYGON ((-3979984.241 4339139.008, -3966502.8...",Умеренный,yellowgreen,3
9,2,"POLYGON ((3214329.108 4464845.851, 3311468.498...",Умеренный,yellowgreen,3


In [9]:
def plot_climate_zones(climate):
    
    layout = Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    
    fig = go.Figure(layout=layout)
    
    for i, vals in tqdm(climate.iterrows()):
        x, y = np.array(vals.geometry.exterior.coords.xy)
        dist = vals['level']
        zone = vals['zone_color']
        zone_color = zone.translate(remove_digits)
        fig.add_trace(go.Scatter(x=x, y=y, text=dist, line_color=zone_color,
                                fill='toself',fillcolor=zone_color, line_width=0.3, 
                                 mode='lines', opacity=0.3))
        
    fig.update_layout(showlegend=False, dragmode='pan', width=1000, height=1000)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False, scaleanchor="x", scaleratio=1)
    
    return fig

In [10]:
# mymap = plot_climate_zones(climate_zones)

In [11]:
# config = dict({'scrollZoom': True})
# mymap.show(config=config)

# russian + climate map

In [12]:
# карта России geojson
ru_map = gpd.read_file('map_data/russia_regions_stats.geojson')
ru_map = ru_map.to_crs('ESRI:102027')

In [13]:
ru_map

,name_ru,name_en,federal_district,count_osm,count_mapbox,mapbox_img_2019_2020_pct,ratio_mapbox_osm,updated,geometry
0,Пензенская область,Penza oblast,Приволжский,255374,300161,34.255814,1.175380,NaN,"MULTIPOLYGON (((-3034722.148 3368565.973, -303..."
1,Саратовская область,Saratov oblast,Приволжский,491455,601512,41.771953,1.223940,NaN,"MULTIPOLYGON (((-3136095.833 3178595.472, -313..."
2,Мордовия,Mordovia,Приволжский,191140,191942,81.121859,1.004200,NaN,"MULTIPOLYGON (((-3000292.487 3408729.339, -300..."
3,Тамбовская область,Tambov oblast,Центральный,240795,184504,58.284703,0.766229,NaN,"MULTIPOLYGON (((-3205780.449 3360578.790, -320..."
4,Рязанская область,Ryazan Oblast,Центральный,135783,238589,87.324990,1.757130,NaN,"MULTIPOLYGON (((-3170823.903 3534078.608, -317..."
...,...,...,...,...,...,...,...,...,...
80,Ивановская область,Ivanovo oblast,Центральный,88559,194870,88.912088,2.200450,NaN,"MULTIPOLYGON (((-2976095.236 3720906.910, -297..."
81,Костромская область,Kostroma oblast,Центральный,77873,135149,43.502731,1.735510,NaN,"MULTIPOLYGON (((-2893325.672 3734902.940, -289..."
82,Марий Эл,Mari El,Приволжский,152951,183194,92.948044,1.197730,NaN,"MULTIPOLYGON (((-2699463.030 3488398.120, -269..."
83,Магаданская область,Magadan oblast,Дальневосточный,9940,13626,47.188903,1.370820,NaN,"MULTIPOLYGON (((2997902.542 4430646.893, 29979..."


In [14]:
# преобразование multipolygon в полигон 
ru_map['area'] = ru_map.geometry.apply(lambda x: x.area)
ru_map_regions = geoseries2shapedict(ru_map.sort_values(by=['area'], 
                                                        axis=0, 
                                                        ascending=False), 
                                     'name_ru', 'rumap')

85it [00:00, 172.73it/s]


In [15]:
def plot_ru_zones(ru_regions, climate):
    
    layout = Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    
    fig = go.Figure(layout=layout)
    
    for reg, vals in tqdm(ru_regions.items()):
        x, y, dist = vals
        fig.add_trace(go.Scatter(x=x, y=y, text=dist, 
                                 line_color='grey', fill='toself',fillcolor='rgba(255,255,255,0)', line_width=1))
    
    for i, vals in tqdm(climate.iterrows()):
        x, y = np.array(vals.geometry.exterior.coords.xy)
        dist = vals['level']
        zone = vals['zone_color']
        zone_color = zone.translate(remove_digits)
        fig.add_trace(go.Scatter(x=x, y=y, text=dist, line_color=zone_color,
                                fill='toself',fillcolor=zone_color, line_width=0.3, 
                                 mode='lines', opacity=0.3))
        
    fig.update_layout(showlegend=False, dragmode='pan', width=1000, height=1000)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False, scaleanchor="x", scaleratio=1)
    
    return fig

In [16]:
# mymap = plot_ru_zones(ru_map_regions, climate_zones)

In [17]:
# config = dict({'scrollZoom': True})
# mymap.show(config=config)

# districts + climate map

In [18]:
all_ru_regions = pd.read_csv('map_data/ru_regions_kadastr.csv')

In [19]:
all_ru_regions = gpd.GeoDataFrame(all_ru_regions, geometry=wkt.loads(all_ru_regions.geometry), crs='ESRI:102027')

In [20]:
all_ru_regions.head(5)

,name,name__ru,wikipedia,geometry,area,reg,dst
0,Селенгинский район,Селенгинский район,ru:Селенгинский район,"POLYGON ((772911.849 2183926.259, 770829.137 2...",6.729938e+09,3.0,18.0
1,Хоринский район,Хоринский район,ru:Хоринский район,"POLYGON ((975660.756 2392376.746, 968958.946 2...",1.216154e+10,3.0,21.0
2,Курумканский район,Курумканский район,ru:Курумканский район,"POLYGON ((1006551.636 2652187.027, 1001762.061...",1.117060e+10,3.0,11.0
3,Брюховецкий район,Брюховецкий район,ru:Брюховецкий район,"POLYGON ((-3642267.062 2820599.306, -3642492.8...",1.132083e+09,23.0,4.0
4,Павловский район,Павловский район,ru:Павловский район (Краснодарский край),"POLYGON ((-3583010.429 2801858.794, -3583399.7...",1.472633e+09,23.0,24.0


In [21]:
all_ru_regions.shape[0]

2365

In [22]:
def plot_reg_zone(climate, regions):
    
    layout = Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    
    fig = go.Figure(layout=layout)
    
    for i, vals in tqdm(climate.iterrows()):
        x, y = np.array(vals.geometry.exterior.coords.xy)
        dist = vals['level']
        zone = vals['zone_color']
        zone_color = zone.translate(remove_digits)
        fig.add_trace(go.Scatter(x=x, y=y, text=dist, line_color=zone_color,
                                fill='toself',fillcolor=zone_color, line_width=0.3, 
                                 mode='lines', opacity=0.3))
        
    for reg, vals in tqdm(regions.items()):
        x, y, dist = vals
        fig.add_trace(go.Scatter(x=x, y=y, text=reg, line_color='black', line_width=0.3, mode='lines'))
        
    fig.update_layout(showlegend=False, dragmode='pan', width=1000, height=1000)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False, scaleanchor="x", scaleratio=1)
    
    return fig

In [23]:
all_ru_regions_map = geoseries2shapedict(all_ru_regions.sort_values(by=['area'], axis=0, ascending=False), 'name', 'regmap')

2365it [00:01, 2306.39it/s]


In [24]:
# mymap = plot_reg_zone(climate_zones, all_ru_regions_map)

In [25]:
# config = dict({'scrollZoom': True})
# mymap.show(config=config)

# districts in climate zones

In [26]:
all_ru_regions[['zone', 'zone_id']] = [None, None]

In [27]:
all_ru_regions.geometry = all_ru_regions.geometry.scale(xfact=0.01, yfact=0.01, origin='center')

In [31]:
for i, all_val in all_ru_regions.iterrows():
    cnter = 0
    list_j = []
    for j, f_val in climate_zones.iterrows():
        try:
            if f_val.geometry.contains(all_val.geometry):
                all_ru_regions.iloc[i,-2:] = [f_val.level, f_val.zone_id]
                list_j.append(j)
                cnter += 1
            else:
                pass
        except:
            print('error, ', i,j)
    if cnter > 1:
        print(cnter, i, list_j)

In [32]:
all_ru_regions.geometry = all_ru_regions.geometry.scale(xfact=100, yfact=100, origin='center')

In [33]:
all_ru_regions.zone.value_counts(normalize=True)

Умеренный         0.979531
Субарктический    0.016205
Арктический       0.004264
Name: zone, dtype: float64

# fires

In [40]:
fire_df = pd.read_csv('map_data/train_raw.csv')
fire_df = gpd.GeoDataFrame(fire_df, geometry=gpd.points_from_xy(fire_df.lon, fire_df.lat), crs='EPSG:4326')
fire_df = fire_df.to_crs('ESRI:102027')

In [41]:
fire_df.type_name.value_counts()

Лесной пожар            223372
Неконтролируемый пал    139880
Контролируемый пал       96870
Природный пожар          87477
Торфяной пожар             164
Name: type_name, dtype: int64

In [42]:
fire_df = fire_df[fire_df.type_name != 'Контролируемый пал']

In [43]:
fire_df

,dt,lon,lat,type_id,type_name,geometry
0,2012-03-13,131.586613,47.866183,4,Природный пожар,POINT (2417438.976 2326194.868)
1,2012-03-13,131.588507,47.880940,4,Природный пожар,POINT (2416940.112 2327608.703)
2,2012-03-13,131.987129,48.497325,3,Лесной пожар,POINT (2415613.703 2395616.579)
3,2012-03-13,131.903087,43.627665,4,Природный пожар,POINT (2614879.276 1946358.175)
4,2012-03-13,131.570614,47.858103,4,Природный пожар,POINT (2416783.637 2325000.347)
...,...,...,...,...,...,...
547753,2021-04-22,131.951770,44.499816,1,Неконтролируемый пал,POINT (2581558.615 2027639.592)
547754,2021-04-22,131.928114,44.846163,3,Лесной пожар,POINT (2565492.064 2058642.903)
547757,2021-04-22,114.285175,51.737231,3,Лесной пожар,POINT (1214981.154 2334769.006)
547759,2021-04-22,119.134877,50.929379,1,Неконтролируемый пал,POINT (1536148.968 2332372.036)


# plot heat

In [44]:
def plot_ru_heat(regions, fires):
    
    # heat_map preparation
    
    fires['x'] = fires.geometry.x
    fires['y'] = fires.geometry.y

    ny = 100
    nx = int(ny*(fires.x.max() - fires.x.min()) / (fires.y.max() - fires.y.min()))
    
    clrscl = [[0, 'rgba(255,128,0,0)'], 
              [0.1, 'rgba(255,128,0,128)'], 
              [0.3, 'rgba(255,128,0,128)'], 
              [0.5, 'rgba(255,64,0,255)'],
              [0.7, 'rgba(255,128,0,128)'],  
              [1, 'rgba(255,0,0,255)']]
    
    layout = Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    
    fig = go.Figure(go.Histogram2d(x=fires.x, y=fires.y,
                                nbinsx=nx,
                                nbinsy=ny,
                                colorscale='hot',
                                showlegend=False, showscale=False, reversescale=True
                            ), layout=layout)
    
    for reg, vals in tqdm(regions.items()):
        x, y, dist = vals
        fig.add_trace(go.Scatter(x=x, y=y, line_color='grey', fill='toself',fillcolor='rgba(255,255,255,0)', line_width=1))

    fig.update_layout(showlegend=False, dragmode='pan', width=1000, height=1000)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False, scaleanchor="x", scaleratio=1)
    
    return fig

In [45]:
# mymap = plot_ru_heat(ru_map_regions, fire_df)

In [46]:
# config = dict({'scrollZoom': True})
# mymap.show(config=config)

# fires in regions

In [47]:
all_ru_regions

,name,name__ru,wikipedia,geometry,area,reg,dst,zone,zone_id
0,Селенгинский район,Селенгинский район,ru:Селенгинский район,"POLYGON ((772911.849 2183926.259, 770829.137 2...",6.729938e+09,3.0,18.0,Умеренный,3
1,Хоринский район,Хоринский район,ru:Хоринский район,"POLYGON ((975660.756 2392376.746, 968958.946 2...",1.216154e+10,3.0,21.0,Умеренный,3
2,Курумканский район,Курумканский район,ru:Курумканский район,"POLYGON ((1006551.636 2652187.027, 1001762.061...",1.117060e+10,3.0,11.0,Умеренный,3
3,Брюховецкий район,Брюховецкий район,ru:Брюховецкий район,"POLYGON ((-3642267.062 2820599.306, -3642492.8...",1.132083e+09,23.0,4.0,Умеренный,3
4,Павловский район,Павловский район,ru:Павловский район (Краснодарский край),"POLYGON ((-3583010.429 2801858.794, -3583399.7...",1.472633e+09,23.0,24.0,Умеренный,3
...,...,...,...,...,...,...,...,...,...
2360,Тамбов,NaN,ru:Тамбов,"POLYGON ((-3124307.505 3303042.861, -3124603.5...",7.601746e+07,68.0,20.0,Умеренный,3
2361,Рассказовский район,NaN,ru:Рассказовский район,"POLYGON ((-3081265.438 3272682.587, -3081149.2...",1.554211e+09,68.0,15.0,Умеренный,3
2362,Сосновский район,NaN,ru:Сосновский район (Тамбовская область),"POLYGON ((-3079943.869 3330265.964, -3082502.7...",2.043358e+09,68.0,18.0,Умеренный,3
2363,Тамбовский район,NaN,ru:Тамбовский район (Тамбовская область),"POLYGON ((-3101331.947 3297711.561, -3101893.6...",2.251141e+09,68.0,20.0,Умеренный,3


In [48]:
all_ru_regions['fire_count'] = None

In [51]:
for i, val in tqdm(all_ru_regions.iterrows(), total = all_ru_regions.shape[0]):
    all_ru_regions.iloc[i, -1] = np.sum(all_ru_regions.iloc[i,3].contains(fire_df.geometry))

100%|██████████████████████████████████████████████████████████████████████████████| 2365/2365 [10:08<00:00,  3.88it/s]


In [52]:
# вычисление отношения количества очагов пожаров к территории
all_ru_regions['area_frac'] = all_ru_regions['area'] / 1e5
all_ru_regions['fires_area_rate'] = all_ru_regions['fire_count']/all_ru_regions['area_frac']
rate_multiply_value = 1 / all_ru_regions['fires_area_rate'].max()
all_ru_regions['fires_area_rate'] = all_ru_regions['fires_area_rate'] * rate_multiply_value
all_ru_regions['fires_area_rate'] = all_ru_regions['fires_area_rate'].apply(lambda x: round(x, 3))
all_ru_regions.drop(columns=['area_frac'], inplace=True)

In [53]:
all_ru_regions

,name,name__ru,wikipedia,geometry,area,reg,dst,zone,zone_id,fire_count,fires_area_rate
0,Селенгинский район,Селенгинский район,ru:Селенгинский район,"POLYGON ((772911.849 2183926.259, 770829.137 2...",6.729938e+09,3.0,18.0,Умеренный,3,224,0.016
1,Хоринский район,Хоринский район,ru:Хоринский район,"POLYGON ((975660.756 2392376.746, 968958.946 2...",1.216154e+10,3.0,21.0,Умеренный,3,529,0.020
2,Курумканский район,Курумканский район,ru:Курумканский район,"POLYGON ((1006551.636 2652187.027, 1001762.061...",1.117060e+10,3.0,11.0,Умеренный,3,1006,0.042
3,Брюховецкий район,Брюховецкий район,ru:Брюховецкий район,"POLYGON ((-3642267.062 2820599.306, -3642492.8...",1.132083e+09,23.0,4.0,Умеренный,3,52,0.022
4,Павловский район,Павловский район,ru:Павловский район (Краснодарский край),"POLYGON ((-3583010.429 2801858.794, -3583399.7...",1.472633e+09,23.0,24.0,Умеренный,3,14,0.004
...,...,...,...,...,...,...,...,...,...,...,...
2360,Тамбов,NaN,ru:Тамбов,"POLYGON ((-3124307.505 3303042.861, -3124603.5...",7.601746e+07,68.0,20.0,Умеренный,3,1,0.006
2361,Рассказовский район,NaN,ru:Рассказовский район,"POLYGON ((-3081265.438 3272682.587, -3081149.2...",1.554211e+09,68.0,15.0,Умеренный,3,44,0.013
2362,Сосновский район,NaN,ru:Сосновский район (Тамбовская область),"POLYGON ((-3079943.869 3330265.964, -3082502.7...",2.043358e+09,68.0,18.0,Умеренный,3,99,0.023
2363,Тамбовский район,NaN,ru:Тамбовский район (Тамбовская область),"POLYGON ((-3101331.947 3297711.561, -3101893.6...",2.251141e+09,68.0,20.0,Умеренный,3,64,0.013


In [57]:
all_ru_regions.sort_values('fires_area_rate', ascending=False).head(10)

,name,name__ru,wikipedia,geometry,area,reg,dst,zone,zone_id,fire_count,fires_area_rate
2236,Находкинский,NaN,ru:Находкинский городской округ,"POLYGON ((2720522.682 1909193.335, 2720512.458...",1.037701e+08,NaN,NaN,Умеренный,3,221,1.000
502,Дальнереченский,NaN,ru:Дальнереченский городской округ,"POLYGON ((2640198.315 2219402.920, 2640133.124...",8.232453e+07,25.0,2.0,Умеренный,3,98,0.559
335,Спасск-Дальний,NaN,ru:Спасск-Дальний,"POLYGON ((2637669.254 2064049.759, 2637525.137...",3.565685e+07,25.0,32.0,Умеренный,3,40,0.527
735,Биробиджан,NaN,ru:Биробиджан,"POLYGON ((2472366.010 2448174.666, 2471980.354...",1.408583e+08,79.0,1.0,Умеренный,3,146,0.487
496,Большой Камень,NaN,ru:Городской округ Большой Камень,"POLYGON ((2674929.548 1909708.834, 2674823.650...",9.689018e+07,NaN,NaN,Умеренный,3,94,0.456
1173,Хорольский район,NaN,ru:Хорольский_район_Приморского_края,"POLYGON ((2616515.340 2048915.551, 2616280.006...",1.606205e+09,25.0,21.0,Умеренный,3,1492,0.436
1070,Ханкайский район,NaN,ru:Ханкайский_район_Приморского_края,"POLYGON ((2599684.053 2049975.439, 2597049.527...",2.224067e+09,25.0,19.0,Умеренный,3,1876,0.396
733,Арсеньевский,NaN,ru:Арсеньевский городской округ,"POLYGON ((2688987.358 2032994.188, 2688611.125...",3.804783e+07,25.0,26.0,Умеренный,3,32,0.395
961,Белогорск,NaN,ru:Городской округ город Белогорск,"POLYGON ((2110955.376 2531680.346, 2110701.491...",9.807114e+07,28.0,2.0,Умеренный,3,80,0.383
1172,Черниговский район,NaN,ru:Черниговский_район_Приморского_края,"POLYGON ((2657760.187 2027637.669, 2657789.463...",1.517187e+09,25.0,22.0,Умеренный,3,1135,0.351


In [31]:
bins = [0, 0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64]
labels = [1,2,3,4,5,6,7]
all_ru_regions['fire_level'] = pd.cut(all_ru_regions['fires_area_rate'], bins=bins, labels=labels)

In [32]:
all_ru_regions

,name,name__ru,wikipedia,geometry,area,reg,dst,fire_count,fires_area_rate,fire_level
0,Селенгинский район,Селенгинский район,ru:Селенгинский район,"POLYGON ((772911.849 2183926.259, 770829.137 2...",6.729938e+09,3.0,18.0,224,0.016,2
1,Хоринский район,Хоринский район,ru:Хоринский район,"POLYGON ((975660.756 2392376.746, 968958.946 2...",1.216154e+10,3.0,21.0,529,0.020,2
2,Курумканский район,Курумканский район,ru:Курумканский район,"POLYGON ((1006551.636 2652187.027, 1001762.061...",1.117060e+10,3.0,11.0,1006,0.042,4
3,Брюховецкий район,Брюховецкий район,ru:Брюховецкий район,"POLYGON ((-3642267.062 2820599.306, -3642492.8...",1.132083e+09,23.0,4.0,52,0.022,3
4,Павловский район,Павловский район,ru:Павловский район (Краснодарский край),"POLYGON ((-3583010.429 2801858.794, -3583399.7...",1.472633e+09,23.0,24.0,14,0.004,1
...,...,...,...,...,...,...,...,...,...,...
2360,Тамбов,NaN,ru:Тамбов,"POLYGON ((-3124307.505 3303042.861, -3124603.5...",7.601746e+07,68.0,20.0,1,0.006,1
2361,Рассказовский район,NaN,ru:Рассказовский район,"POLYGON ((-3081265.438 3272682.587, -3081149.2...",1.554211e+09,68.0,15.0,44,0.013,2
2362,Сосновский район,NaN,ru:Сосновский район (Тамбовская область),"POLYGON ((-3079943.869 3330265.964, -3082502.7...",2.043358e+09,68.0,18.0,99,0.023,3
2363,Тамбовский район,NaN,ru:Тамбовский район (Тамбовская область),"POLYGON ((-3101331.947 3297711.561, -3101893.6...",2.251141e+09,68.0,20.0,64,0.013,2


# plot all

In [50]:
def plot_reg_zone(climate, fires, na_regions):
    
    fires['x'] = fires.geometry.x
    fires['y'] = fires.geometry.y

    ny = 100
    nx = int(ny*(fires.x.max() - fires.x.min()) / (fires.y.max() - fires.y.min()))
    
    layout = Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
    
    fig = go.Figure(go.Histogram2d(x=fires.x, y=fires.y,
                                nbinsx=nx,
                                nbinsy=ny,
                                colorscale='hot',
                                showlegend=False, showscale=False, reversescale=True
                            ), layout=layout)
    
    for i, vals in tqdm(climate.iterrows()):
        x, y = np.array(vals.geometry.exterior.coords.xy)
        dist = vals['level']
        zone = vals['zone_color']
        zone_color = zone.translate(remove_digits)
        fig.add_trace(go.Scatter(x=x, y=y, text=dist, line_color=zone_color,
                                fill='toself',fillcolor=zone_color, line_width=0.3, 
                                 mode='lines', opacity=0.3))
        
    for reg, vals in tqdm(na_regions.items()):
        x, y, dist = vals
        fig.add_trace(go.Scatter(x=x, y=y, text=reg, line_color='black', line_width=0.3, mode='lines'))
#     for reg, vals in tqdm(ru_regions.items()):
#         x, y, dist = vals
#         fig.add_trace(go.Scatter(x=x, y=y, text=dist, 
#                                  line_color='grey', fill='toself',fillcolor='rgba(255,255,255,0)', line_width=1))
        
    fig.update_layout(showlegend=False, dragmode='pan', width=1000, height=1000)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False, scaleanchor="x", scaleratio=1)
    
    return fig

In [51]:
all_ru_regions_map = geoseries2shapedict(all_ru_regions.sort_values(by=['area'], axis=0, ascending=False), 'name', 'regmap')

2365it [00:01, 2271.19it/s]


In [53]:
# mymap = plot_reg_zone(climate_zones, fire_df, all_ru_regions_map)

12it [00:00, 912.05it/s]
100%|████████████████████████████████████████████████████████████████████████████| 2220/2220 [00:01<00:00, 1897.33it/s]


In [58]:
# config = dict({'scrollZoom': True})
# mymap.show(config=config)